# Julia is a modern dynamic language

  * sophisticated type system
  * rich set of built-in types (numeric and general-purpose)
  * user-defined types
  * metaprogramming (macros)


## Built-in numeric types

Julia's built-in numeric types include a wide range of 
  * integers: Int16, Int32, Int64 (and unsigned ints), and arbitrary-precision BigInts 
  * floating-points: Float16, Float32, Float64, and arbitrary-precision BigFloats
  * rationals using the integer types
  * complex numbers formed from above
  * vectors, matrices, linear algebra on above

## Integers

In [ ]:
typeof(7)

## Real and complex numbers

In [ ]:
π

In [ ]:
typeof(π)

In [ ]:
Float64(π)

In [ ]:
BigFloat(π)

In [ ]:
typeof(ans)

In [ ]:
pi = BigFloat(π)

In [ ]:
setprecision(512)

In [ ]:
pi = BigFloat(π)

In [ ]:
exp(pi*1im)

In [ ]:
typeof(ans)

In [ ]:
4 + 5im

In [ ]:
im^2

In [ ]:
typeof(ans)

## Rational numbers

In [ ]:
5//2

In [ ]:
Float64(5//2)

In [ ]:
typeof(ans)

In [ ]:
3//4 + 5//7

In [ ]:
Rational(Float32(pi))

In [ ]:
@show Float32(13176795/4194304);
@show Float32(π);

In [ ]:
5.0 + 3im

In [ ]:
typeof(ans)

In [ ]:
5//3 + 7//9im

In [ ]:
typeof(ans)

## Introspection on Julia's numeric type hierarchy

In [ ]:
subtypes(Number)

In [ ]:
subtypes(Real)

In [ ]:
subtypes(AbstractFloat)

In [ ]:
Int32 <: Number    # are Int64's numbers?

In [ ]:
Int64 <: Real      # are Int64's a subset of the reals?

In [ ]:
Int64 <: Float64   # are Int64's a subset of 64-bit floating-point numbers?

In [ ]:
17 ∈ 0:3:21        # is 17 an element of the set 0,3,6,9,12,15,18,21? (a set operation)

In [ ]:
typeof(ans)

## Solve a system of rational equations

illustrates linear algebra over arbitrary-precision rationals

In [ ]:
A = convert(Matrix{Rational{Int64}}, rand(1:10,5,5))/10

In [ ]:
x = [3//4; 17//3; -1//13; -7//11 ; 3//19]
b = A*x

In [ ]:
x̂ = A\b

In [ ]:
x - x̂

Note that Julia's backslash operator (and LU decomp) works over all its numeric types

Float32 and Float64 LU, QR, SVD, etc. are calls to LAPACK

## Define a new numeric type: scalar finite field GF(p)

example from Andreas Noack, CSAIL MIT http://andreasnoack.github.io/talks/2015AprilStanford_AndreasNoack.ipynb

#### define scalar finite field GF(p) i.e. integers modulo p

In [1]:
# Scalar finite fields. P is the modulus, T is the integer type (Int16, Int32, ...)
immutable GF{P,T} <: Number where {P,T<:Integer}
    data::T
    function GF{P,T}(x::Integer) where {P,T<:Integer}
        return new(mod(x, P))
    end
end
GF{P}(x::T) where {P,T<:Integer} = GF{P,T}(x)

 immutable: In Julia variables change values, but numbers do not.

In [2]:
# basic methods for scalar finite field
import Base: convert, inv, one, promote_rule, show, zero

function call{P}(::Type{GF{P}}, x::Integer)
    if !isprime(P)
        throw(ArgumentError("P must be a prime"))
    end
    return GF{P,typeof(x)}(mod(x, P))
end
convert{P,T}(::Type{GF{P,T}}, x::Integer) = GF{P}(x)
convert{P}(::Type{GF{P}}, x::Integer) = GF{P}(x)
convert{P,T}(::Type{GF{P,T}}, x::GF{P}) = GF{P,T}(x.data)
promote_rule{P,T1,T2<:Integer}(::Type{GF{P,T1}}, ::Type{T2}) = GF{P,promote_type(T1,T2)}
show(io::IO, x::GF) = show(io, x.data)

show (generic function with 268 methods)

 ### Define basic arithmetic operations using metaprogramming


In [3]:
import Base: +, -, *, /

for op in (:+, :-, :*)
    @eval begin
        ($op){P,T}(x::GF{P,T}, y::GF{P,T}) = GF{P,T}($(op)(x.data, y.data))
    end
end

### Create a couple variables of type GF(5) , do some arithmetic

In [4]:
x, y = GF{5}(9), GF{5}(8)
@show x
@show y
@show x + y
@show x - y
@show x * y
;

x = 4
y = 3
x + y = 2
x - y = 1
x * y = 2


In [5]:
# Division requires slightly more care
function inv{P,T}(x::GF{P,T})
    if x == zero(x)
        throw(DivideError())
    end
    r, u, v = gcdx(x.data, P)
    GF{P,T}(u)
end
(/){P}(x::GF{P}, y::GF{P}) = x*inv(y)
;

In [6]:
@show x / y
@show x \ y # backslash on any Number is defined in terms of /, so we get it autmomatically
;

x / y = 3
x \ y = 2


### With the field operations defined, we can now do linear algebra

In [ ]:
# create 4x4 matrix of random GF(5) elems
srand(1234)
A = [GF{5}(rand(0:4)) for i = 1:4, j = 1:4] 

In [ ]:
b = [GF{5}(rand(0:4)) for i = 1:4]

In [ ]:
x̂ = A\b

##### Whoa! The built-in matrix backslash works because the generic Julia LU decomp code works over any field

In [ ]:
A*x̂ - b

In [ ]:
typeof(x)

In [ ]:
methods(+)

In [ ]:
@which x + y

In [ ]:
@which GF{7}(4) + GF{7}(1)

In [ ]:
@which 4+8

### Compare computational costs of GF(p) to Float64 

matrix-matrix mult 

In [ ]:
A1, A2 = rand(1:100, 100, 100), rand(1:100, 100, 100)
A1*A2 # warm up to be sure function is compiled
print("int mat mult   ")
@time A1*A2

AF1, AF2 = map(GF{5}, A1), map(GF{5}, A2)
AF1*AF2
print("GF(p) mat mult ")
@time AF1*AF2
;

LU factorization: Float64 via LAPACK, GF(p) via generic LU algorithm

In [ ]:
lufact(A1)
print("Float64 mat lufact ")

@time lufact(A1) # Promoted to Float64 and calls LAPACK

F = lufact(AF1,Val{false})
while F.info != 0
    AF1[F.info, F.info] += 1
    F = lufact(AF1, Val{false})
end

lufact(AF1)
print("GF(p) mat lufact   ")
@time lufact(AF1) # Non-blocked generic LU implemented in Julia
;

#### According to Noack, the increased cost of GF(p) over floats is just the modulus operations 

In [ ]:
@code_lowered(lufact(A1))

In [ ]:
@code_lowered(lufact(AF1))

#### maybe show symbolic mathematics in Noack's notebook...